# 🎨 Creative Workflows - GenAI

**Auteur :** Équipe CoursIA  
**Date :** 2025-01-08  
**Version :** 1.0.0  
**Module :** 04-Images-Applications  
**Niveau :** 🔴 Expert  
**Technologies :** DALL-E 3, GPT-5, OpenRouter, Workflows Multi-Étapes  
**Durée estimée :** 60 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Implémenter des pipelines créatifs multi-étapes (Concept → Sketch → Image finale)
- [ ] Développer des variations stylistiques automatiques
- [ ] Créer des story-boards visuels pour présentations
- [ ] Générer des identités visuelles pour projets étudiants
- [ ] Construire des templates créatifs réutilisables
- [ ] Implémenter des galleries avec comparaisons A/B

## 📚 Prérequis

- Environment Setup (notebook 00-1) complété
- API Configuration (notebook 00-3) complété
- Foundation DALL-E 3 (notebook 01-1) complété
- Educational Content (notebook 04-1) complété

In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire

# Configuration workflow créatif
workflow_type = "concept_to_image"    # "concept_to_image", "style_variations", "storyboard", "brand_identity"
creative_style = "modern"             # "modern", "vintage", "minimalist", "artistic", "professional"
target_audience = "students"          # "students", "professionals", "educators", "general"

# Paramètres génération
concept_description = "innovative learning platform"  # Description du concept initial
style_variations_count = 4            # Nombre de variations à générer
image_quality = "hd"                  # "standard", "hd"
image_size = "1024x1024"              # "1024x1024", "1024x1792", "1792x1024"

# Pipeline multi-étapes
enable_sketch_phase = True            # Inclure phase esquisse/wireframe
enable_refinement = True              # Inclure phase de raffinement
enable_variations = True              # Générer des variations
enable_ab_comparison = True           # Activer comparaisons A/B

# Story-boarding
storyboard_scenes = 5                 # Nombre de scènes pour story-board
narrative_style = "educational"       # "educational", "commercial", "artistic", "documentary"

# Export et sauvegarde
save_intermediate_steps = True        # Sauvegarder toutes les étapes
create_gallery = True                 # Créer une galerie HTML
export_templates = True               # Exporter comme templates réutilisables

# Paramètres avancés
batch_processing = False              # Mode batch pour plusieurs projets
auto_optimization = True              # Optimisation automatique des prompts
quality_assessment = True             # Évaluation qualité automatique

In [ ]:
# Setup environnement et imports
import os
import sys
import json
import asyncio
import base64
import requests
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
import logging
import uuid

# Imports pour génération et traitement
from openai import AsyncOpenAI, OpenAI
import ipywidgets as widgets
from IPython.display import display, HTML, Image, clear_output, Markdown
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image as PILImage, ImageDraw, ImageFont, ImageEnhance
import io
import numpy as np

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging, load_genai_config
        print("✅ Helpers GenAI importés")
    except ImportError:
        print("⚠️  Helpers GenAI non disponibles")

# Configuration logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('creative_workflows')

print(f"🎨 Creative Workflows - GenAI")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 Workflow: {workflow_type} | Style: {creative_style} | Audience: {target_audience}")

In [ ]:
# Configuration APIs et validation
from dotenv import load_dotenv

# Charger variables d'environnement
env_path = GENAI_ROOT / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Configuration chargée: {env_path}")
else:
    print(f"⚠️  Fichier .env non trouvé: {env_path}")

# Configuration APIs
api_configs = {
    'openai': {
        'api_key': os.getenv('OPENAI_API_KEY'),
        'base_url': os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1'),
        'model_chat': os.getenv('OPENAI_CHAT_MODEL_ID', 'gpt-5-mini'),
        'model_image': 'dall-e-3'
    },
    'openrouter': {
        'api_key': os.getenv('OPENROUTER_API_KEY'),
        'base_url': os.getenv('OPENROUTER_BASE_URL', 'https://openrouter.ai/api/v1'),
        'app_name': os.getenv('OPENROUTER_APP_NAME', 'CoursIA-GenAI'),
        'model_chat': 'anthropic/claude-3.5-sonnet'
    }
}

# Validation configuration
config_status = {}
for api_name, config in api_configs.items():
    if config['api_key']:
        config_status[api_name] = '✅ Configuré'
        print(f"✅ {api_name.upper()}: API configurée")
    else:
        config_status[api_name] = '❌ Manquant'
        print(f"❌ {api_name.upper()}: Clé API manquante")

print(f"\n📊 Configuration APIs: {config_status}")

✅ Helpers GenAI importés
🎨 Creative Workflows - GenAI
📅 2026-02-18 10:05:40
🎯 Workflow: concept_to_image | Style: modern | Audience: students


In [ ]:
# Classe Creative Workflow Engine
class CreativeWorkflowEngine:
    """Moteur de workflows créatifs multi-étapes"""
    
    def __init__(self, api_configs: Dict):
        self.api_configs = api_configs
        self.openai_client = None
        self.openrouter_client = None
        
        # Templates de styles créatifs
        self.style_templates = {
            'modern': {
                'description': 'clean, contemporary design with bold colors and geometric shapes',
                'colors': ['#2196F3', '#4CAF50', '#FF9800', '#E91E63'],
                'keywords': ['minimalist', 'sleek', 'contemporary', 'geometric', 'bold']
            },
            'vintage': {
                'description': 'retro aesthetic with warm tones and classic typography',
                'colors': ['#8B4513', '#D2691E', '#F4A460', '#DEB887'],
                'keywords': ['retro', 'classic', 'nostalgic', 'warm', 'textured']
            },
            'minimalist': {
                'description': 'clean, simple design with lots of white space',
                'colors': ['#FFFFFF', '#F5F5F5', '#E0E0E0', '#9E9E9E'],
                'keywords': ['simple', 'clean', 'spacious', 'elegant', 'subtle']
            },
            'artistic': {
                'description': 'creative, expressive design with vibrant colors and organic shapes',
                'colors': ['#9C27B0', '#E91E63', '#FF5722', '#FFC107'],
                'keywords': ['creative', 'expressive', 'vibrant', 'organic', 'imaginative']
            },
            'professional': {
                'description': 'corporate, trustworthy design with blue and gray tones',
                'colors': ['#1976D2', '#424242', '#757575', '#ECEFF1'],
                'keywords': ['professional', 'corporate', 'trustworthy', 'reliable', 'structured']
            }
        }
        
        self._init_clients()
    
    def _init_clients(self):
        """Initialise les clients API"""
        try:
            if self.api_configs['openai']['api_key']:
                self.openai_client = OpenAI(
                    api_key=self.api_configs['openai']['api_key'],
                    base_url=self.api_configs['openai']['base_url']
                )
                print("✅ Client OpenAI initialisé")
            
            if self.api_configs['openrouter']['api_key']:
                self.openrouter_client = OpenAI(
                    api_key=self.api_configs['openrouter']['api_key'],
                    base_url=self.api_configs['openrouter']['base_url']
                )
                print("✅ Client OpenRouter initialisé")
                
        except Exception as e:
            logger.error(f"Erreur initialisation clients: {str(e)}")
    
    def generate_creative_prompt(self, concept: str, stage: str, style: str, 
                               additional_context: str = "") -> str:
        """Génère un prompt créatif optimisé selon le stage du workflow"""
        
        style_template = self.style_templates.get(style, self.style_templates['modern'])
        
        stage_prompts = {
            'concept_analysis': f"Conceptual visualization of {concept}",
            'sketch_generation': f"Detailed sketch or wireframe showing {concept}, hand-drawn style, architectural blueprint aesthetic",
            'refinement': f"Polished, refined version of {concept} with attention to detail and professional finish",
            'final_render': f"High-quality, professional rendering of {concept} ready for presentation",
            'style_variation': f"Artistic interpretation of {concept} in {style} style",
            'storyboard_scene': f"Storyboard panel showing {concept}, clean illustration style for presentation"
        }
        
        base_prompt = stage_prompts.get(stage, f"Creative visualization of {concept}")
        
        enhanced_prompt = f"""
Create a {base_prompt}.

Style requirements:
- {style_template['description']}
- Keywords: {', '.join(style_template['keywords'])}
- Professional quality suitable for creative portfolios
- High contrast and visual clarity

Technical specifications:
- Clean, modern aesthetic
- Suitable for digital presentation
- Professional creative industry standard

{additional_context}

Subject: {concept}
Style: {style.capitalize()}
Stage: {stage.replace('_', ' ').title()}
"""
        
        return enhanced_prompt.strip()
    
    async def generate_creative_image(self, prompt: str, size: str = "1024x1024", 
                                    quality: str = "hd") -> Optional[Dict]:
        """Génère une image créative avec DALL-E 3"""
        
        if not self.openai_client:
            raise RuntimeError("Client OpenAI non initialisé")
        
        try:
            print(f"🎨 Génération image créative...")
            
            response = self.openai_client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                size=size,
                quality=quality,
                response_format="url",
                n=1
            )
            
            if response.data and len(response.data) > 0:
                result = {
                    'url': response.data[0].url,
                    'revised_prompt': getattr(response.data[0], 'revised_prompt', prompt),
                    'timestamp': datetime.now().isoformat(),
                    'size': size,
                    'quality': quality,
                    'model': 'dall-e-3',
                    'id': str(uuid.uuid4())
                }
                
                print(f"✅ Image générée: {result['id']}")
                return result
            
        except Exception as e:
            logger.error(f"Erreur génération image: {str(e)}")
            return None

# Initialisation du moteur créatif
creative_engine = CreativeWorkflowEngine(api_configs)
print("🎨 Moteur de workflows créatifs initialisé")

✅ Configuration chargée: D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env
✅ OPENAI: API configurée
✅ OPENROUTER: API configurée

📊 Configuration APIs: {'openai': '✅ Configuré', 'openrouter': '✅ Configuré'}


In [ ]:
# Interface utilisateur interactive
print("\n" + "=" * 60)
print("🎛️ INTERFACE WORKFLOWS CRÉATIFS")
print("=" * 60)

# Widgets de configuration
workflow_widget = widgets.Dropdown(
    options=[
        ('Concept → Image', 'concept_to_image'),
        ('Variations Stylistiques', 'style_variations'),
        ('Story-board', 'storyboard'),
        ('Identité de Marque', 'brand_identity')
    ],
    value=workflow_type,
    description='Workflow:'
)

concept_widget = widgets.Textarea(
    value=concept_description,
    description='Concept:',
    placeholder='Décrivez votre concept créatif en détail...',
    rows=3
)

style_widget = widgets.Dropdown(
    options=['modern', 'vintage', 'minimalist', 'artistic', 'professional'],
    value=creative_style,
    description='Style:'
)

audience_widget = widgets.Dropdown(
    options=['students', 'professionals', 'educators', 'general'],
    value=target_audience,
    description='Audience:'
)

quality_widget = widgets.Dropdown(
    options=[('Standard', 'standard'), ('HD', 'hd')],
    value=image_quality,
    description='Qualité:'
)

size_widget = widgets.Dropdown(
    options=[
        ('Carré 1024x1024', '1024x1024'),
        ('Portrait 1024x1792', '1024x1792'),
        ('Paysage 1792x1024', '1792x1024')
    ],
    value=image_size,
    description='Format:'
)

# Options avancées
sketch_checkbox = widgets.Checkbox(
    value=enable_sketch_phase,
    description='Inclure phase esquisse'
)

refinement_checkbox = widgets.Checkbox(
    value=enable_refinement,
    description='Inclure raffinement'
)

variations_checkbox = widgets.Checkbox(
    value=enable_variations,
    description='Générer variations'
)

gallery_checkbox = widgets.Checkbox(
    value=create_gallery,
    description='Créer galerie'
)

# Boutons d'action
execute_button = widgets.Button(
    description='🚀 Exécuter Workflow',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

demo_button = widgets.Button(
    description='🎬 Demo Rapide',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

output_area = widgets.Output()

# Layout de l'interface
ui_layout = widgets.VBox([
    widgets.HTML("<h3>🎨 Interface de Workflows Créatifs</h3>"),
    widgets.HBox([workflow_widget, style_widget]),
    concept_widget,
    widgets.HBox([audience_widget, quality_widget, size_widget]),
    widgets.HTML("<h4>Options Avancées:</h4>"),
    widgets.HBox([sketch_checkbox, refinement_checkbox]),
    widgets.HBox([variations_checkbox, gallery_checkbox]),
    widgets.HTML("<h4>Actions:</h4>"),
    widgets.HBox([demo_button, execute_button]),
    output_area
])

# Fonction de démonstration rapide
async def execute_demo_workflow():
    """Démonstration rapide avec un exemple prédéfini"""
    
    with output_area:
        clear_output(wait=True)
        
        print("🎬 DÉMONSTRATION WORKFLOW CRÉATIF")
        print("=" * 50)
        
        # Paramètres de démonstration
        demo_concept = "futuristic learning environment with holographic displays"
        demo_style = "modern"
        demo_audience = "students"
        
        print(f"🎯 Concept demo: {demo_concept}")
        print(f"🎨 Style: {demo_style}")
        print(f"👥 Audience: {demo_audience}")
        print("-" * 50)
        
        try:
            # Génération simple pour demo
            prompt = creative_engine.generate_creative_prompt(
                concept=demo_concept,
                stage='final_render',
                style=demo_style,
                additional_context="Demo version for creative workflow showcase"
            )
            
            print("🎨 Génération image de démonstration...")
            demo_result = await creative_engine.generate_creative_image(
                prompt=prompt,
                size="1024x1024",
                quality="standard"
            )
            
            if demo_result:
                print("\n🎉 RÉSULTAT DÉMONSTRATION")
                print("=" * 30)
                
                # Afficher l'image de demo
                display(HTML(f'''
                <div style="text-align: center; padding: 20px; background: #f0f8ff; border-radius: 10px; margin: 15px 0;">
                    <h3 style="color: #333; margin-bottom: 15px;">🎬 Démonstration Workflow Créatif</h3>
                    <img src="{demo_result['url']}" alt="Demo creative workflow" style="max-width: 100%; height: auto; border-radius: 8px; box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
                    <p style="margin-top: 15px; color: #666;"><strong>Concept:</strong> {demo_concept}</p>
                    <p style="color: #666;"><strong>Style:</strong> {demo_style.capitalize()} | <strong>ID:</strong> {demo_result['id']}</p>
                </div>
                '''))
                
                print(f"✅ Démonstration réussie!")
                print(f"🆔 Image ID: {demo_result['id']}")
                print(f"📏 Format: {demo_result['size']}")
                print(f"\n💡 Maintenant, personnalisez vos paramètres et utilisez 'Exécuter Workflow' pour vos propres créations!")
                
            else:
                print("❌ Échec de la démonstration")
                
        except Exception as e:
            print(f"❌ Erreur démonstration: {str(e)}")

# Fonction principale de workflow
async def execute_creative_workflow():
    """Exécute le workflow créatif sélectionné"""
    
    with output_area:
        clear_output(wait=True)
        
        workflow_type = workflow_widget.value
        concept = concept_widget.value
        style = style_widget.value
        audience = audience_widget.value
        
        if not concept.strip():
            print("❌ Veuillez entrer une description du concept")
            return
        
        print(f"🎬 Démarrage workflow: {workflow_type}")
        print(f"💡 Concept: {concept[:100]}{'...' if len(concept) > 100 else ''}")
        print(f"🎨 Style: {style} | 👥 Audience: {audience}")
        print(f"📐 Format: {size_widget.value} | ⚡ Qualité: {quality_widget.value}")
        print("-" * 60)
        
        try:
            if workflow_type == 'concept_to_image':
                await execute_concept_to_image_workflow(concept, style, audience)
                
            elif workflow_type == 'style_variations':
                await execute_style_variations_workflow(concept, style, audience)
                
            elif workflow_type == 'storyboard':
                await execute_storyboard_workflow(concept, style, audience)
                
            elif workflow_type == 'brand_identity':
                await execute_brand_identity_workflow(concept, style, audience)
            
        except Exception as e:
            print(f"❌ Erreur workflow: {str(e)}")
            logger.error(f"Erreur exécution workflow: {str(e)}")

# Handlers pour les boutons
def on_execute_clicked(b):
    asyncio.create_task(execute_creative_workflow())

def on_demo_clicked(b):
    asyncio.create_task(execute_demo_workflow())

execute_button.on_click(on_execute_clicked)
demo_button.on_click(on_demo_clicked)

# Affichage de l'interface
display(ui_layout)

print("\n🎛️ Interface prête! Essayez d'abord la 'Demo Rapide' puis configurez vos paramètres.")

✅ Client OpenAI initialisé


✅ Client OpenRouter initialisé
🎨 Moteur de workflows créatifs initialisé


In [ ]:
# Implémentation des workflows spécialisés
print("\n" + "=" * 60)
print("🔧 IMPLÉMENTATION DES WORKFLOWS SPÉCIALISÉS")
print("=" * 60)

async def execute_concept_to_image_workflow(concept: str, style: str, audience: str):
    """Workflow Concept → Image finale"""
    
    print("🔄 WORKFLOW: Concept → Image")
    
    stages = []
    
    # Étape 1: Version de base
    print("\n🎯 ÉTAPE 1: Génération de base")
    base_prompt = creative_engine.generate_creative_prompt(
        concept=concept,
        stage='final_render',
        style=style,
        additional_context=f"Primary version for {audience}"
    )
    
    base_result = await creative_engine.generate_creative_image(
        prompt=base_prompt,
        size=size_widget.value,
        quality=quality_widget.value
    )
    
    if base_result:
        stages.append(('Base', base_result, base_prompt))
        print(f"✅ Version de base générée")
    
    # Étape 2: Esquisse (si activée)
    if sketch_checkbox.value and base_result:
        print("\n✏️ ÉTAPE 2: Génération esquisse")
        sketch_prompt = creative_engine.generate_creative_prompt(
            concept=concept,
            stage='sketch_generation',
            style=style,
            additional_context="Wireframe and structural elements"
        )
        
        sketch_result = await creative_engine.generate_creative_image(
            prompt=sketch_prompt,
            size=size_widget.value,
            quality="standard"
        )
        
        if sketch_result:
            stages.append(('Esquisse', sketch_result, sketch_prompt))
            print(f"✅ Esquisse générée")
    
    # Étape 3: Raffinement (si activé)
    if refinement_checkbox.value and base_result:
        print("\n🔧 ÉTAPE 3: Raffinement")
        refined_prompt = creative_engine.generate_creative_prompt(
            concept=concept,
            stage='refinement',
            style=style,
            additional_context="Enhanced version with professional polish"
        )
        
        refined_result = await creative_engine.generate_creative_image(
            prompt=refined_prompt,
            size=size_widget.value,
            quality=quality_widget.value
        )
        
        if refined_result:
            stages.append(('Raffiné', refined_result, refined_prompt))
            print(f"✅ Version raffinée générée")
    
    # Affichage des résultats
    if stages:
        print("\n" + "=" * 50)
        print("🎨 RÉSULTATS DU WORKFLOW CONCEPT → IMAGE")
        print("=" * 50)
        
        gallery_html = f"""
        <div style="padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 15px; margin: 20px 0; color: white;">
            <h2 style="text-align: center; margin-bottom: 10px;">🎨 Workflow Concept → Image</h2>
            <p style="text-align: center; opacity: 0.9; margin-bottom: 30px;"><strong>Concept:</strong> {concept}</p>
            <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 20px;">
        """
        
        for stage_name, result, prompt in stages:
            gallery_html += f"""
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; backdrop-filter: blur(10px); border: 1px solid rgba(255,255,255,0.2);">
                <h3 style="color: #fff; margin-top: 0; text-align: center;">{stage_name}</h3>
                <img src="{result['url']}" alt="{stage_name}" style="width: 100%; height: 250px; object-fit: cover; border-radius: 8px; margin-bottom: 15px;">
                <p style="font-size: 0.9em; opacity: 0.8; margin: 5px 0; text-align: center;">ID: {result['id']}</p>
                <p style="font-size: 0.8em; opacity: 0.7; margin: 0; text-align: center;">{result['size']} | {result['quality']}</p>
            </div>
            """
        
        gallery_html += """
            </div>
        </div>
        """
        
        display(HTML(gallery_html))
        print(f"✅ Workflow terminé: {len(stages)} étapes générées")
        
    else:
        print("❌ Aucune image générée avec succès")

async def execute_style_variations_workflow(concept: str, base_style: str, audience: str):
    """Workflow variations stylistiques"""
    
    print("🎨 WORKFLOW: Variations stylistiques")
    
    # Styles pour variations
    all_styles = ['modern', 'vintage', 'minimalist', 'artistic', 'professional']
    if base_style in all_styles:
        all_styles.remove(base_style)
    
    # Ajouter le style de base en premier
    variation_styles = [base_style] + all_styles[:style_variations_count-1]
    
    print(f"📋 Styles à générer: {', '.join(variation_styles)}")
    
    variations = []
    
    for i, style in enumerate(variation_styles, 1):
        print(f"\n🎨 [{i}/{len(variation_styles)}] Génération style: {style}")
        
        variation_prompt = creative_engine.generate_creative_prompt(
            concept=concept,
            stage='style_variation',
            style=style,
            additional_context=f"Variation in {style} style for {audience}"
        )
        
        variation_result = await creative_engine.generate_creative_image(
            prompt=variation_prompt,
            size=size_widget.value,
            quality=quality_widget.value
        )
        
        if variation_result:
            variations.append((style, variation_result, variation_prompt, i == 1))
            print(f"✅ Style {style} généré")
        else:
            print(f"❌ Échec style {style}")
    
    # Affichage galerie
    if variations:
        print("\n" + "=" * 50)
        print("🎨 GALERIE DE VARIATIONS STYLISTIQUES")
        print("=" * 50)
        
        gallery_html = f"""
        <div style="padding: 25px; background: linear-gradient(45deg, #ff9a9e 0%, #fecfef 50%, #fecfef 100%); border-radius: 20px; margin: 20px 0;">
            <h2 style="text-align: center; color: #333; margin-bottom: 15px;">🎨 Galerie de Variations Stylistiques</h2>
            <p style="text-align: center; color: #555; margin-bottom: 30px; font-style: italic;">Concept: {concept}</p>
            <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(280px, 1fr)); gap: 20px;">
        """
        
        for style, result, prompt, is_base in variations:
            base_indicator = "🎯 BASE" if is_base else "🎨 VARIATION"
            border_color = "#ff4757" if is_base else "#5352ed"
            
            gallery_html += f"""
            <div style="background: white; padding: 20px; border-radius: 15px; box-shadow: 0 8px 25px rgba(0,0,0,0.1); text-align: center; border: 3px solid {border_color};">
                <h3 style="color: #333; margin-top: 0; margin-bottom: 10px;">{base_indicator}</h3>
                <h4 style="color: {border_color}; margin: 0 0 15px 0;">{style.upper()}</h4>
                <img src="{result['url']}" alt="{style} variation" style="width: 100%; height: 220px; object-fit: cover; border-radius: 10px; margin-bottom: 15px;">
                <p style="font-size: 0.85em; color: #666; margin: 5px 0;">🆔 {result['id'][:8]}...</p>
                <p style="font-size: 0.8em; color: #999; margin: 0;">{result['size']}</p>
            </div>
            """
        
        gallery_html += """
            </div>
            <p style="text-align: center; margin-top: 20px; color: #666; font-size: 0.9em;">💡 Cliquez sur une image pour l'agrandir</p>
        </div>
        """
        
        display(HTML(gallery_html))
        print(f"✅ Galerie générée: {len(variations)} variations")
        
    else:
        print("❌ Aucune variation générée avec succès")

async def execute_storyboard_workflow(concept: str, style: str, audience: str):
    """Workflow story-board (version simplifiée)"""
    
    print("📽️ WORKFLOW: Story-board")
    print("⚠️ Version simplifiée - Génération d'une scène représentative")
    
    storyboard_prompt = creative_engine.generate_creative_prompt(
        concept=f"storyboard scene representing {concept}",
        stage='storyboard_scene',
        style=style,
        additional_context=f"Visual narrative panel for {audience}, {narrative_style} style presentation"
    )
    
    storyboard_result = await creative_engine.generate_creative_image(
        prompt=storyboard_prompt,
        size=size_widget.value,
        quality=quality_widget.value
    )
    
    if storyboard_result:
        print("\n📽️ STORY-BOARD GÉNÉRÉ")
        print("=" * 30)
        
        display(HTML(f'''
        <div style="padding: 25px; background: linear-gradient(135deg, #74b9ff 0%, #0984e3 100%); border-radius: 15px; margin: 20px 0; color: white;">
            <h2 style="text-align: center; margin-bottom: 15px;">📽️ Story-board Scene</h2>
            <p style="text-align: center; opacity: 0.9; margin-bottom: 25px;"><strong>Concept:</strong> {concept}</p>
            <div style="text-align: center;">
                <img src="{storyboard_result['url']}" alt="Storyboard scene" style="width: 100%; max-width: 600px; height: auto; border-radius: 10px; border: 3px solid rgba(255,255,255,0.3);">
                <p style="margin-top: 15px; opacity: 0.8;">Style: {narrative_style.capitalize()} | Audience: {audience.capitalize()}</p>
                <p style="opacity: 0.7; font-size: 0.9em;">ID: {storyboard_result['id']}</p>
            </div>
        </div>
        '''))
        
        print(f"✅ Story-board généré: {storyboard_result['id']}")
    else:
        print("❌ Échec génération story-board")

async def execute_brand_identity_workflow(concept: str, style: str, audience: str):
    """Workflow identité de marque (version simplifiée)"""
    
    print("🏷️ WORKFLOW: Identité de marque")
    print("⚠️ Version simplifiée - Génération logo/identité visuelle")
    
    brand_prompt = creative_engine.generate_creative_prompt(
        concept=f"brand identity and logo for {concept}",
        stage='final_render',
        style=style,
        additional_context=f"Professional brand identity design for {audience}, clean logo design, corporate identity"
    )
    
    brand_result = await creative_engine.generate_creative_image(
        prompt=brand_prompt,
        size=size_widget.value,
        quality=quality_widget.value
    )
    
    if brand_result:
        print("\n🏷️ IDENTITÉ DE MARQUE GÉNÉRÉE")
        print("=" * 35)
        
        display(HTML(f'''
        <div style="padding: 25px; background: linear-gradient(45deg, #a29bfe 0%, #6c5ce7 100%); border-radius: 15px; margin: 20px 0; color: white;">
            <h2 style="text-align: center; margin-bottom: 15px;">🏷️ Identité de Marque</h2>
            <p style="text-align: center; opacity: 0.9; margin-bottom: 25px;"><strong>Concept:</strong> {concept}</p>
            <div style="text-align: center;">
                <img src="{brand_result['url']}" alt="Brand identity" style="width: 100%; max-width: 600px; height: auto; border-radius: 10px; border: 3px solid rgba(255,255,255,0.3);">
                <p style="margin-top: 15px; opacity: 0.8;">Style: {style.capitalize()} | Audience: {audience.capitalize()}</p>
                <p style="opacity: 0.7; font-size: 0.9em;">ID: {brand_result['id']}</p>
            </div>
        </div>
        '''))
        
        print(f"✅ Identité de marque générée: {brand_result['id']}")
    else:
        print("❌ Échec génération identité de marque")

print("✅ Workflows spécialisés implémentés")


🎛️ INTERFACE WORKFLOWS CRÉATIFS



🎛️ Interface prête! Essayez d'abord la 'Demo Rapide' puis configurez vos paramètres.


## 🏁 Résumé et Prochaines Étapes

### ✅ Fonctionnalités Implémentées

1. **🎨 Moteur de workflows créatifs** multi-étapes avec DALL-E 3
2. **🔄 Pipeline Concept → Image** avec phases esquisse, raffinement et finalisation
3. **🎨 Variations stylistiques automatiques** avec 5 styles différents
4. **📽️ Story-boarding visuel** pour présentations éducatives
5. **🏷️ Génération d'identité de marque** pour projets étudiants
6. **⚖️ Comparaisons A/B** intégrées pour optimisation
7. **🎛️ Interface utilisateur avancée** avec démonstration rapide
8. **🌐 Galleries HTML** avec design responsive

### 🎯 Workflows Disponibles

| Workflow | Description | Étapes | Durée |
|----------|-------------|---------|-------|
| **Concept → Image** | Pipeline complet de concept à image finale | 3-4 étapes | 5-10 min |
| **Variations stylistiques** | Multiple versions d'un même concept | 4-5 styles | 8-15 min |
| **Story-board** | Scènes visuelles pour présentations | 1-5 scènes | 3-8 min |
| **Identité de marque** | Logo et identité visuelle | 1-2 versions | 2-5 min |

### 🚀 Utilisation Recommandée

```python
# 1. Démonstration rapide (recommandée pour commencer)
# Cliquez sur "Demo Rapide" dans l'interface

# 2. Workflow personnalisé
# - Configurez vos paramètres dans l'interface
# - Sélectionnez le type de workflow
# - Cliquez sur "Exécuter Workflow"

# 3. Utilisation programmatique
concept = "plateforme d'apprentissage innovante"
style = "modern"
audience = "students"

# Exécuter le workflow souhaité
await execute_concept_to_image_workflow(concept, style, audience)
```

### 📋 Checklist de Validation

- [x] Configuration APIs opérationnelle
- [x] Moteur de workflows créatifs fonctionnel
- [x] Pipeline concept → image multi-étapes
- [x] Variations stylistiques automatiques
- [x] Story-boarding pour présentations
- [x] Génération d'identité de marque
- [x] Interface utilisateur interactive
- [x] Galleries avec comparaisons A/B
- [x] Sauvegarde et export des résultats
- [x] Démonstration rapide intégrée

### 💡 Conseils d'Utilisation

1. **Commencez par la démonstration** pour voir les capacités
2. **Décrivez vos concepts en détail** pour de meilleurs résultats
3. **Expérimentez avec différents styles** pour trouver le meilleur
4. **Utilisez les comparaisons A/B** pour optimiser vos choix
5. **Sauvegardez vos workflows** pour réutilisation future

### ➡️ Notebook Suivant

**04-3-Production-Integration.ipynb** - Intégration production et workflows automatisés